# CFD Validator Agent

Agents can automate verification steps that are costly or subjective:

compute PDE residuals and check physics consistency,

compare PINN outputs to trusted solvers or learned surrogates,

spot anomalies, outliers, or boundary-condition violations,

produce human-readable verdicts and evidence for on-chain contracts.
This Agent validates the Simulation performed by PhysicNemo by using rule-based checks


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(8,4)
print('torch', torch.__version__, 'cuda', torch.cuda.is_available())

In [ ]:
## Settings
USE_PRECOMPUTED=False
MODEL_PATH='pinn_airfoil_naca0012.pth'
PRECOMP_U='airfoil_u.npy'
PRECOMP_V='airfoil_v.npy'
PRECOMP_P='airfoil_p.npy'
PRECOMP_RHO='airfoil_rho.npy'
AIRFOIL_COORDS='naca0012_coords.csv'
xmin, xmax, ymin, ymax = -1.5, 3.0, -1.5, 1.5
nx, ny = 300, 150
xs = np.linspace(xmin, xmax, nx)
ys = np.linspace(ymin, ymax, ny)
X, Y = np.meshgrid(xs, ys)
grid = np.vstack([X.ravel(), Y.ravel()]).T


In [ ]:
## Load or evaluate model (user must provide model or precomputed arrays)
import os
if USE_PRECOMPUTED and os.path.exists(PRECOMP_P):
    u = np.load(PRECOMP_U); v = np.load(PRECOMP_V); p = np.load(PRECOMP_P)
    print('Loaded precomputed fields')
else:
    if os.path.exists(MODEL_PATH):
        print('Model file exists — notebook will try to load it. Ensure network class matches saved model.')
    else:
        raise RuntimeError('No model or precomputed fields found. Set USE_PRECOMPUTED or provide MODEL_PATH.')


In [ ]:
## Validators
from scipy import interpolate

def check_bc_airfoil(u_grid, v_grid, airfoil_coords, xs, ys):
    f_u = interpolate.RegularGridInterpolator((ys, xs), u_grid)
    f_v = interpolate.RegularGridInterpolator((ys, xs), v_grid)
    uv = np.array([f_u((pt[1], pt[0])) for pt in airfoil_coords]).flatten()
    vv = np.array([f_v((pt[1], pt[0])) for pt in airfoil_coords]).flatten()
    pts = airfoil_coords
    tangents = np.zeros_like(pts)
    for i in range(len(pts)):
        i0=(i-1)%len(pts); i1=(i+1)%len(pts)
        tg = pts[i1]-pts[i0]
        tg = tg/(np.linalg.norm(tg)+1e-12)
        tangents[i]=tg
    normals = np.zeros_like(tangents); normals[:,0]=-tangents[:,1]; normals[:,1]=tangents[:,0]
    vdotn = uv*normals[:,0] + vv*normals[:,1]
    return np.mean(np.abs(vdotn)), np.max(np.abs(vdotn))

def compute_pressure_forces(p_grid, airfoil_coords, xs, ys):
    f_p = interpolate.RegularGridInterpolator((ys, xs), p_grid)
    p_surf = np.array([f_p((pt[1], pt[0])) for pt in airfoil_coords]).flatten()
    pts = airfoil_coords
    tangents = np.zeros_like(pts)
    for i in range(len(pts)):
        i0=(i-1)%len(pts); i1=(i+1)%len(pts)
        tg = pts[i1]-pts[i0]
        tg = tg/(np.linalg.norm(tg)+1e-12)
        tangents[i]=tg
    normals = np.zeros_like(tangents); normals[:,0] = -tangents[:,1]; normals[:,1] = tangents[:,0]
    seg_lengths = np.linalg.norm(np.roll(pts, -1, axis=0) - pts, axis=1)
    force_segments = (-p_surf[:,None] * normals) * seg_lengths[:,None]
    F_total = force_segments.sum(axis=0)
    Drag = -F_total[0]; Lift = -F_total[1]
    q_inf = 0.5 * 1.225 * (1.2 * ((1.4*287.058*288.15)**0.5))**2
    Cl = Lift / (q_inf * 1.0); Cd = Drag / (q_inf * 1.0)
    return {'Lift': float(Lift), 'Drag': float(Drag), 'Cl': float(Cl), 'Cd': float(Cd)}

print('Validator helpers ready')


In [ ]:
## Report generation helper
import matplotlib.pyplot as plt

def save_report_text(report, filename='cfd_validation_report.pdf'):
    fig, ax = plt.subplots(figsize=(8.27,11.69))
    ax.axis('off')
    text = '\n'.join([f'{k}: {v}' for k,v in report.items()])
    ax.text(0.01, 0.99, text, va='top', ha='left', fontsize=10, family='monospace')
    fig.savefig(filename)
    print('Saved report to', filename)
    plt.close(fig)


## Usage example
1. Set `USE_PRECOMPUTED=True` and save your `u,v,p` grids as numpy arrays named above and run this notebook.
2. Or paste the evaluation code that computes `u_grid,v_grid,p_grid` into the 'Load or evaluate model' cell so the notebook produces fields.

Once fields are available you can run the validators and generate the report.